## Topic Modeling

This notebook use Latent Dirichlet Allocation (LDA) to topic model the documents, then project them to 2 dimensions for display. 

In [1]:
# Import packages 
from os import listdir
import os
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
import re
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from gensim import corpora, models
from nltk.stem.porter import *
stemmer = PorterStemmer()
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


[nltk_data] Downloading package stopwords to /Users/jshen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jshen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Preprocessing

In [5]:
# Lowercases everything 
# Removes stopwords and words with 3 or fewer chars 
# Stems and lemmatizes all words

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [8]:
f = open("./full_text/759082635-MIT.pdf.txt", "r")
print(preprocess(f.read()))

['teacher', 'self', 'efficaci', 'cape', 'town', 'approach', 'enhanc', 'qualiti', 'educ', 'yeseul', 'submit', 'depart', 'urban', 'studi', 'plan', 'partial', 'fulfil', 'requir', 'degre', 'master', 'citi', 'plan', 'massachusett', 'institut', 'technolog', 'archiv', 'massachusett', 'institut', 'cnolog', 'june', 'yeseul', 'right', 'reserv', 'author', 'grant', 'permiss', 'reproduc', 'distribut', 'publicli', 'paper', 'electron', 'copi', 'thesi', 'document', 'author', 'yeseul', 'depart', 'urban', 'studi', 'plan', 'accept', 'professor', 'ceasar', 'mcdowel', 'depart', 'urban', 'studi', 'plan', 'thesi', 'supervisor', 'professor', 'joseph', 'ferreira', 'depart', 'urban', 'studi', 'plan', 'chair', 'committe', 'certifi', 'teacher', 'self', 'efficaci', 'cape', 'town', 'approach', 'enhanc', 'qualiti', 'educ', 'yeseul', 'submit', 'depart', 'urban', 'studi', 'plan', 'partial', 'fulfil', 'requir', 'degre', 'master', 'citi', 'plan', 'abstract', 'person', 'teacher', 'self', 'efficaci', 'belief', 'abil', 'ov

In [9]:
class Documents(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            if ".pdf.txt" in fname:
                f = open(os.path.join(self.dirname, fname), "r")

                yield f.read().split("\n")
        
    def __len__(self):
        return len([x for x in os.listdir(self.dirname) if ".pdf.txt" in x])
 
documents = Documents('full_text_processed') 

In [6]:
# dictionary.save("dict.dict")
dictionary = corpora.Dictionary.load('dict.dict')

2023-01-29 16:57:55,079 : INFO : loading Dictionary object from dict.dict
2023-01-29 16:57:55,148 : INFO : Dictionary lifecycle event {'fname': 'dict.dict', 'datetime': '2023-01-29T16:57:55.148120', 'gensim': '4.3.0', 'python': '3.10.6 (main, Aug 30 2022, 05:12:36) [Clang 13.1.6 (clang-1316.0.21.2.5)]', 'platform': 'macOS-12.5.1-x86_64-i386-64bit', 'event': 'loaded'}


In [11]:
class BagOfWords(object):
    def __init__(self, documents, dictionary):
        self.documents = documents
        self.dict = dictionary
 
    def __iter__(self):
        for doc in self.documents: 
            yield self.dict.doc2bow(doc)
            
    def __len__(self): 
        return len(self.documents)
 
bow = BagOfWords(documents, dictionary) 

## Tuning
Let's determine how many passes and topics we need.

In [13]:
# Make an index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

num_topics = 10
passes = 10

In [13]:
from gensim.models import CoherenceModel

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(bow))  

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=documents, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

As can be seen, perplexity has largely converged so 10 passes should be enough. Let's determine the correct number of topics to choose now. We will run the topic model with 10, 15, 20, ... 10 topics, optimizing for coherence.

In [14]:
topic_sizes = list(range(10, 100, 5))
coherences = []

for n in topic_sizes: 
    print("topic model w", n, "topics")

    lda_model = gensim.models.LdaModel(
        corpus=bow,
        id2word=id2word,
        alpha='auto',
        eta='auto',
        num_topics=n,
        passes=passes,
        eval_every=None
    )
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=documents, dictionary=dictionary, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('Coherence Score: ', coherence_lda)
    coherences.append(coherence_lda)
    

2023-01-09 11:53:57,688 : INFO : using autotuned alpha, starting with [0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668, 0.016666668]
2023-01-09 11:53:57,706 : INFO : using serial LDA version on this node


topic model w 60 topics


2023-01-09 11:53:58,599 : INFO : running online (multi-pass) LDA training, 60 topics, 10 passes over the supplied corpus of 58520 documents, updating model once every 2000 documents, evaluating perplexity every 0 documents, iterating 50x with a convergence threshold of 0.001000
2023-01-09 11:54:08,672 : INFO : PROGRESS: pass 0, at document #2000/58520
2023-01-09 11:54:21,099 : INFO : optimized alpha [0.029188093, 0.02875347, 0.02874849, 0.028926833, 0.029122084, 0.029351223, 0.029197399, 0.029199567, 0.0292673, 0.029506352, 0.029015116, 0.02878464, 0.029445572, 0.029285977, 0.029148057, 0.029138185, 0.029859966, 0.029339254, 0.029017894, 0.028978363, 0.029513188, 0.029356906, 0.028968718, 0.029038943, 0.028852891, 0.028748821, 0.028872281, 0.029569296, 0.02875647, 0.029251773, 0.029242072, 0.02923778, 0.029230308, 0.028766315, 0.029402684, 0.02962542, 0.029377842, 0.029418742, 0.028836466, 0.029341854, 0.02894814, 0.028916376, 0.029558474, 0.029835857, 0.029557325, 0.029382017, 0.02912

2023-01-09 11:55:32,436 : INFO : topic #41 (0.032): 0.016*"price" + 0.014*"market" + 0.009*"demand" + 0.009*"econom" + 0.009*"invest" + 0.009*"polici" + 0.008*"industri" + 0.007*"vehicl" + 0.006*"sector" + 0.006*"emiss"
2023-01-09 11:55:32,439 : INFO : topic #44 (0.033): 0.029*"user" + 0.013*"file" + 0.010*"game" + 0.007*"interfac" + 0.005*"player" + 0.005*"video" + 0.005*"server" + 0.005*"particip" + 0.005*"display" + 0.005*"softwar"
2023-01-09 11:55:32,449 : INFO : topic diff=2.678240, rho=0.500000
2023-01-09 11:55:42,545 : INFO : PROGRESS: pass 0, at document #10000/58520
2023-01-09 11:55:54,583 : INFO : optimized alpha [0.023595858, 0.026604956, 0.023100393, 0.02504971, 0.025963647, 0.030971482, 0.022566132, 0.02480298, 0.027057491, 0.028680671, 0.030594906, 0.027785532, 0.03249, 0.024249211, 0.02471272, 0.024661778, 0.023830673, 0.023697298, 0.02653525, 0.025701817, 0.023973072, 0.02914553, 0.031067358, 0.024754435, 0.025243327, 0.024321822, 0.022981739, 0.024810705, 0.02536107, 0

2023-01-09 11:57:04,550 : INFO : topic #31 (0.033): 0.028*"imag" + 0.018*"train" + 0.013*"cluster" + 0.011*"algorithm" + 0.011*"network" + 0.010*"dataset" + 0.008*"detect" + 0.007*"segment" + 0.007*"neural" + 0.007*"visual"
2023-01-09 11:57:04,553 : INFO : topic #44 (0.035): 0.040*"user" + 0.011*"interfac" + 0.010*"file" + 0.007*"display" + 0.007*"game" + 0.007*"softwar" + 0.006*"text" + 0.006*"particip" + 0.006*"visual" + 0.005*"video"
2023-01-09 11:57:04,555 : INFO : topic #12 (0.035): 0.025*"algorithm" + 0.012*"matrix" + 0.012*"theorem" + 0.011*"proof" + 0.011*"vector" + 0.010*"graph" + 0.009*"random" + 0.009*"lemma" + 0.008*"network" + 0.007*"iter"
2023-01-09 11:57:04,569 : INFO : topic diff=2.919734, rho=0.353553
2023-01-09 11:57:14,879 : INFO : PROGRESS: pass 0, at document #18000/58520
2023-01-09 11:57:28,437 : INFO : optimized alpha [0.020410713, 0.025178963, 0.020693503, 0.022696337, 0.025431734, 0.030280853, 0.02070847, 0.02338034, 0.02523351, 0.026549114, 0.031427197, 0.0283

...

2023-01-09 15:54:08,633 : INFO : running online (multi-pass) LDA training, 65 topics, 10 passes over the supplied corpus of 58520 documents, updating model once every 2000 documents, evaluating perplexity every 0 documents, iterating 50x with a convergence threshold of 0.001000
2023-01-09 15:54:18,705 : INFO : PROGRESS: pass 0, at document #2000/58520
2023-01-09 15:54:30,520 : INFO : optimized alpha [0.026942555, 0.02666945, 0.026583778, 0.026687872, 0.02644869, 0.027053867, 0.026943063, 0.026650343, 0.027153824, 0.026808934, 0.026553644, 0.027039569, 0.02691847, 0.027050171, 0.026757415, 0.02692039, 0.02689687, 0.027248941, 0.026801907, 0.02694964, 0.027160175, 0.026994895, 0.027066264, 0.02667878, 0.026493039, 0.02721332, 0.026567083, 0.02700369, 0.027311543, 0.02670569, 0.02710982, 0.0264654, 0.027356686, 0.026884563, 0.026544407, 0.026701935, 0.027049415, 0.027303629, 0.026592996, 0.026881887, 0.026855852, 0.027143802, 0.027130427, 0.027054396, 0.026407322, 0.026892532, 0.026946813

2023-01-09 15:55:40,503 : INFO : topic #2 (0.022): 0.019*"fiber" + 0.016*"sound" + 0.014*"frequenc" + 0.014*"hair" + 0.010*"stimul" + 0.008*"bead" + 0.007*"auditori" + 0.007*"cochlear" + 0.007*"stereocilia" + 0.007*"nois"
2023-01-09 15:55:40,507 : INFO : topic #16 (0.030): 0.014*"temperatur" + 0.012*"film" + 0.010*"devic" + 0.009*"layer" + 0.006*"heat" + 0.006*"thermal" + 0.006*"particl" + 0.005*"contact" + 0.005*"substrat" + 0.005*"silicon"
2023-01-09 15:55:40,509 : INFO : topic #4 (0.031): 0.012*"industri" + 0.011*"compani" + 0.009*"market" + 0.009*"servic" + 0.009*"busi" + 0.007*"govern" + 0.006*"econom" + 0.005*"invest" + 0.005*"polici" + 0.005*"firm"
2023-01-09 15:55:40,511 : INFO : topic #25 (0.032): 0.025*"user" + 0.017*"imag" + 0.010*"game" + 0.007*"file" + 0.006*"algorithm" + 0.006*"server" + 0.006*"interfac" + 0.006*"player" + 0.005*"visual" + 0.005*"video"
2023-01-09 15:55:40,522 : INFO : topic diff=2.909546, rho=0.500000
2023-01-09 15:55:51,385 : INFO : PROGRESS: pass 0, at

2023-01-09 15:57:15,566 : INFO : merging changes from 2000 documents into a model of 58520 documents
2023-01-09 15:57:15,818 : INFO : topic #20 (0.019): 0.086*"machin" + 0.046*"cut" + 0.043*"wear" + 0.016*"workpiec" + 0.015*"eros" + 0.014*"scratch" + 0.009*"contact" + 0.009*"runner" + 0.008*"finish" + 0.007*"lubric"
2023-01-09 15:57:15,821 : INFO : topic #23 (0.019): 0.029*"iron" + 0.013*"carbon" + 0.012*"splice" + 0.010*"ocean" + 0.010*"marin" + 0.008*"martensit" + 0.007*"abund" + 0.006*"cell" + 0.006*"cultur" + 0.006*"phytoplankton"
2023-01-09 15:57:15,823 : INFO : topic #13 (0.030): 0.032*"frequenc" + 0.022*"signal" + 0.021*"wave" + 0.015*"puls" + 0.012*"nois" + 0.012*"magnet" + 0.010*"mode" + 0.010*"beam" + 0.009*"amplitud" + 0.008*"probe"
2023-01-09 15:57:15,825 : INFO : topic #4 (0.031): 0.017*"industri" + 0.016*"compani" + 0.015*"busi" + 0.012*"servic" + 0.011*"market" + 0.008*"govern" + 0.008*"econom" + 0.007*"firm" + 0.007*"innov" + 0.006*"countri"
2023-01-09 15:57:15,827 : IN

2023-01-09 15:59:01,400 : INFO : merging changes from 2000 documents into a model of 58520 documents
2023-01-09 15:59:01,663 : INFO : topic #23 (0.018): 0.028*"iron" + 0.016*"carbon" + 0.011*"marin" + 0.009*"ocean" + 0.009*"microbi" + 0.009*"abund" + 0.009*"nutrient" + 0.008*"speci" + 0.008*"growth" + 0.007*"bacteria"
2023-01-09 15:59:01,668 : INFO : topic #1 (0.018): 0.035*"vowel" + 0.030*"tone" + 0.021*"segment" + 0.017*"syllabl" + 0.016*"thruster" + 0.016*"voic" + 0.015*"pitch" + 0.015*"speech" + 0.013*"stress" + 0.013*"conson"
2023-01-09 15:59:01,671 : INFO : topic #39 (0.032): 0.014*"architectur" + 0.007*"cultur" + 0.005*"histori" + 0.005*"centuri" + 0.005*"citi" + 0.004*"modern" + 0.004*"hous" + 0.004*"live" + 0.004*"architect" + 0.003*"museum"
2023-01-09 15:59:01,674 : INFO : topic #4 (0.033): 0.021*"industri" + 0.019*"compani" + 0.015*"busi" + 0.014*"market" + 0.012*"servic" + 0.011*"firm" + 0.010*"govern" + 0.009*"innov" + 0.008*"econom" + 0.007*"countri"
2023-01-09 15:59:01,6

KeyboardInterrupt: 

* 10 topics: 0.6612879085854633 
* 15 topics: 0.6639288994005009
* 20 topics: 0.7000287678468625
* 25 topics:, 0.6915824547608532
* 30 topics: 0.6924143718277599
* 35 topics: 0.701246357671909
* 40 topics: 0.6965632313596052
* 45 topics: 0.6989573216919259
* 50 topics: 0.7013887893321301
* 55 topics: 0.7124166808223836
* 60 topics: 0.701336791986567

Since it has started to plateau, no need to try higher topics. Let's use 20.

In [14]:
lda_model = gensim.models.LdaModel(
        corpus=bow,
        id2word=id2word,
        alpha='auto',
        eta='auto',
        num_topics=20,
        passes=passes,
        eval_every=None
    )

2023-01-26 17:19:52,927 : INFO : using autotuned alpha, starting with [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]
2023-01-26 17:19:52,946 : INFO : using serial LDA version on this node
2023-01-26 17:19:53,492 : INFO : running online (multi-pass) LDA training, 20 topics, 10 passes over the supplied corpus of 58520 documents, updating model once every 2000 documents, evaluating perplexity every 0 documents, iterating 50x with a convergence threshold of 0.001000
2023-01-26 17:20:03,725 : INFO : PROGRESS: pass 0, at document #2000/58520
2023-01-26 17:20:12,842 : INFO : optimized alpha [0.08768579, 0.087081164, 0.08684851, 0.08742098, 0.08761066, 0.08807154, 0.08800642, 0.087809235, 0.08784145, 0.08831928, 0.08765544, 0.087107316, 0.08795489, 0.08808571, 0.08742876, 0.08784047, 0.08907269, 0.08781302, 0.087540284, 0.0875987]
2023-01-26 17:20:12,909 : INFO : merging changes from 2000 documents into a model of 58520 

2023-01-26 17:21:32,071 : INFO : topic #5 (0.072): 0.013*"frequenc" + 0.010*"imag" + 0.008*"signal" + 0.008*"wave" + 0.007*"nois" + 0.006*"optic" + 0.006*"mode" + 0.006*"simul" + 0.005*"temperatur" + 0.005*"puls"
2023-01-26 17:21:32,075 : INFO : topic diff=0.988574, rho=0.447214
2023-01-26 17:21:42,227 : INFO : PROGRESS: pass 0, at document #12000/58520
2023-01-26 17:21:51,237 : INFO : optimized alpha [0.041645307, 0.051837552, 0.040142216, 0.0625539, 0.05175725, 0.06786021, 0.044664007, 0.047252145, 0.057185996, 0.057899125, 0.06199894, 0.0606282, 0.067969084, 0.051238652, 0.047272354, 0.04454296, 0.04836987, 0.04471391, 0.054962218, 0.05635403]
2023-01-26 17:21:51,303 : INFO : merging changes from 2000 documents into a model of 58520 documents
2023-01-26 17:21:51,378 : INFO : topic #2 (0.040): 0.011*"fiber" + 0.007*"filament" + 0.007*"polym" + 0.007*"nanotub" + 0.007*"plasma" + 0.005*"simul" + 0.005*"reaction" + 0.005*"chain" + 0.005*"temperatur" + 0.005*"diffus"
2023-01-26 17:21:51,

2023-01-26 17:23:12,678 : INFO : topic diff=0.830221, rho=0.316228
2023-01-26 17:23:24,787 : INFO : PROGRESS: pass 0, at document #22000/58520
2023-01-26 17:23:34,255 : INFO : optimized alpha [0.03294121, 0.04373423, 0.032356847, 0.059325356, 0.046678975, 0.05838703, 0.03594354, 0.03910289, 0.0493748, 0.047498755, 0.057351906, 0.057288736, 0.0642282, 0.04672926, 0.037836872, 0.03655054, 0.040414274, 0.035876535, 0.049927756, 0.048677877]
2023-01-26 17:23:34,326 : INFO : merging changes from 2000 documents into a model of 58520 documents
2023-01-26 17:23:34,401 : INFO : topic #2 (0.032): 0.020*"fiber" + 0.014*"polym" + 0.011*"diffus" + 0.010*"chain" + 0.010*"flame" + 0.010*"simul" + 0.008*"molecul" + 0.007*"molecular" + 0.006*"particl" + 0.006*"nanotub"
2023-01-26 17:23:34,404 : INFO : topic #0 (0.033): 0.013*"neuron" + 0.012*"sentenc" + 0.010*"brain" + 0.007*"argument" + 0.006*"movement" + 0.005*"languag" + 0.005*"verb" + 0.004*"semant" + 0.004*"stimulu" + 0.004*"predic"
2023-01-26 17:

2023-01-26 17:25:05,627 : INFO : PROGRESS: pass 0, at document #32000/58520
2023-01-26 17:25:15,010 : INFO : optimized alpha [0.030426264, 0.04089721, 0.030847572, 0.061110042, 0.047471974, 0.05673661, 0.034889877, 0.038101483, 0.048544474, 0.04660956, 0.05758223, 0.05898933, 0.06600798, 0.046567302, 0.035203073, 0.035283662, 0.039541937, 0.032607436, 0.04966211, 0.04684597]
2023-01-26 17:25:15,075 : INFO : merging changes from 2000 documents into a model of 58520 documents
2023-01-26 17:25:15,150 : INFO : topic #0 (0.030): 0.011*"sentenc" + 0.010*"languag" + 0.009*"neuron" + 0.008*"movement" + 0.007*"brain" + 0.007*"argument" + 0.007*"verb" + 0.006*"claus" + 0.006*"phrase" + 0.006*"semant"
2023-01-26 17:25:15,152 : INFO : topic #2 (0.031): 0.018*"fiber" + 0.012*"polym" + 0.011*"strain" + 0.011*"simul" + 0.010*"diffus" + 0.008*"molecul" + 0.008*"chain" + 0.008*"particl" + 0.007*"molecular" + 0.007*"membran"
2023-01-26 17:25:15,154 : INFO : topic #11 (0.059): 0.014*"load" + 0.009*"angl"

2023-01-26 17:26:56,528 : INFO : optimized alpha [0.030186266, 0.039802656, 0.030938532, 0.06435941, 0.05021516, 0.055116817, 0.034955777, 0.03839041, 0.049669515, 0.04819021, 0.05979543, 0.0614352, 0.068431675, 0.04758645, 0.033563837, 0.035886653, 0.04103801, 0.031515207, 0.050377287, 0.047130924]
2023-01-26 17:26:56,594 : INFO : merging changes from 2000 documents into a model of 58520 documents
2023-01-26 17:26:56,674 : INFO : topic #0 (0.030): 0.011*"sentenc" + 0.010*"languag" + 0.009*"verb" + 0.009*"neuron" + 0.007*"argument" + 0.007*"brain" + 0.007*"movement" + 0.006*"claus" + 0.006*"head" + 0.005*"semant"
2023-01-26 17:26:56,676 : INFO : topic #2 (0.031): 0.016*"polym" + 0.015*"fiber" + 0.014*"strain" + 0.010*"particl" + 0.009*"simul" + 0.009*"diffus" + 0.007*"chain" + 0.007*"stress" + 0.007*"bond" + 0.006*"composit"
2023-01-26 17:26:56,678 : INFO : topic #11 (0.061): 0.013*"load" + 0.009*"angl" + 0.008*"motion" + 0.008*"vehicl" + 0.008*"sensor" + 0.007*"actuat" + 0.007*"motor"

...

2023-01-26 18:51:06,126 : INFO : topic #12 (0.103): 0.022*"algorithm" + 0.012*"matrix" + 0.010*"vector" + 0.010*"random" + 0.009*"proof" + 0.008*"theorem" + 0.008*"graph" + 0.008*"network" + 0.007*"simul" + 0.007*"bind"
2023-01-26 18:51:06,130 : INFO : topic diff=0.031619, rho=0.159597
2023-01-26 18:51:16,633 : INFO : PROGRESS: pass 9, at document #16000/58520
2023-01-26 18:51:26,600 : INFO : optimized alpha [0.038189355, 0.03931411, 0.062778614, 0.100118496, 0.09630156, 0.058577053, 0.05335795, 0.081066, 0.07378092, 0.08546707, 0.08067092, 0.0825141, 0.10246701, 0.074260645, 0.032051105, 0.06552438, 0.0796392, 0.03608534, 0.07209459, 0.064782746]
2023-01-26 18:51:26,667 : INFO : merging changes from 2000 documents into a model of 58520 documents
2023-01-26 18:51:26,742 : INFO : topic #14 (0.032): 0.013*"rock" + 0.008*"fault" + 0.007*"depth" + 0.007*"zone" + 0.007*"earth" + 0.007*"soil" + 0.007*"sediment" + 0.007*"mantl" + 0.006*"melt" + 0.006*"composit"
2023-01-26 18:51:26,744 : INFO 

2023-01-26 18:52:56,149 : INFO : topic diff=0.031175, rho=0.159597
2023-01-26 18:53:06,808 : INFO : PROGRESS: pass 9, at document #26000/58520
2023-01-26 18:53:19,225 : INFO : optimized alpha [0.038000487, 0.039156724, 0.063119814, 0.10059093, 0.09651348, 0.05821629, 0.053749293, 0.08174288, 0.07394065, 0.08462819, 0.08041428, 0.0815649, 0.102248274, 0.07392157, 0.03209808, 0.06539235, 0.07924281, 0.035999656, 0.07189444, 0.06532278]
2023-01-26 18:53:19,293 : INFO : merging changes from 2000 documents into a model of 58520 documents
2023-01-26 18:53:19,369 : INFO : topic #14 (0.032): 0.013*"rock" + 0.009*"fault" + 0.008*"depth" + 0.007*"zone" + 0.007*"earth" + 0.007*"soil" + 0.006*"composit" + 0.006*"geolog" + 0.006*"seismic" + 0.006*"melt"
2023-01-26 18:53:19,371 : INFO : topic #17 (0.036): 0.046*"cell" + 0.019*"protein" + 0.013*"gene" + 0.008*"bind" + 0.006*"sequenc" + 0.005*"tissu" + 0.005*"regul" + 0.005*"transcript" + 0.005*"biolog" + 0.005*"cultur"
2023-01-26 18:53:19,373 : INFO 

2023-01-26 18:54:51,706 : INFO : PROGRESS: pass 9, at document #36000/58520
2023-01-26 18:55:00,781 : INFO : optimized alpha [0.038126998, 0.03930337, 0.06343047, 0.10067734, 0.09650263, 0.05819039, 0.05393423, 0.08178151, 0.074275605, 0.0846351, 0.07991643, 0.08185104, 0.10231943, 0.07391556, 0.032268375, 0.06529704, 0.07868265, 0.03584217, 0.07206077, 0.0653621]
2023-01-26 18:55:00,853 : INFO : merging changes from 2000 documents into a model of 58520 documents
2023-01-26 18:55:00,953 : INFO : topic #14 (0.032): 0.014*"rock" + 0.009*"fault" + 0.008*"zone" + 0.008*"depth" + 0.007*"earth" + 0.007*"soil" + 0.007*"sediment" + 0.006*"geolog" + 0.006*"composit" + 0.006*"mantl"
2023-01-26 18:55:00,955 : INFO : topic #17 (0.036): 0.046*"cell" + 0.018*"protein" + 0.013*"gene" + 0.009*"bind" + 0.006*"sequenc" + 0.005*"tissu" + 0.005*"regul" + 0.005*"tumor" + 0.005*"human" + 0.005*"biolog"
2023-01-26 18:55:00,957 : INFO : topic #4 (0.097): 0.017*"compani" + 0.014*"custom" + 0.013*"industri" + 0

2023-01-26 18:56:40,846 : INFO : merging changes from 2000 documents into a model of 58520 documents
2023-01-26 18:56:40,924 : INFO : topic #14 (0.032): 0.014*"rock" + 0.010*"fault" + 0.008*"zone" + 0.008*"depth" + 0.008*"soil" + 0.007*"earth" + 0.007*"clay" + 0.007*"seismic" + 0.007*"earthquak" + 0.006*"miner"
2023-01-26 18:56:40,926 : INFO : topic #17 (0.036): 0.044*"cell" + 0.019*"protein" + 0.013*"gene" + 0.009*"bind" + 0.007*"sequenc" + 0.005*"regul" + 0.005*"biolog" + 0.005*"human" + 0.005*"tissu" + 0.005*"transcript"
2023-01-26 18:56:40,928 : INFO : topic #4 (0.096): 0.016*"compani" + 0.013*"industri" + 0.013*"custom" + 0.012*"manufactur" + 0.012*"busi" + 0.011*"team" + 0.011*"servic" + 0.010*"market" + 0.010*"decis" + 0.007*"strategi"
2023-01-26 18:56:40,930 : INFO : topic #12 (0.101): 0.023*"algorithm" + 0.012*"matrix" + 0.010*"proof" + 0.009*"random" + 0.009*"vector" + 0.009*"theorem" + 0.008*"graph" + 0.008*"network" + 0.007*"lemma" + 0.007*"bind"
2023-01-26 18:56:40,932 : I

2023-01-26 18:58:19,711 : INFO : topic #14 (0.032): 0.013*"rock" + 0.010*"fault" + 0.008*"zone" + 0.007*"depth" + 0.007*"soil" + 0.007*"melt" + 0.007*"earth" + 0.007*"clay" + 0.007*"mantl" + 0.006*"composit"
2023-01-26 18:58:19,713 : INFO : topic #17 (0.036): 0.045*"cell" + 0.019*"protein" + 0.012*"gene" + 0.009*"bind" + 0.007*"sequenc" + 0.005*"tissu" + 0.005*"human" + 0.005*"tumor" + 0.005*"regul" + 0.005*"biolog"
2023-01-26 18:58:19,715 : INFO : topic #4 (0.097): 0.017*"compani" + 0.013*"custom" + 0.013*"industri" + 0.012*"manufactur" + 0.012*"busi" + 0.011*"team" + 0.011*"servic" + 0.009*"market" + 0.009*"decis" + 0.007*"strategi"
2023-01-26 18:58:19,717 : INFO : topic #12 (0.101): 0.023*"algorithm" + 0.011*"matrix" + 0.010*"random" + 0.009*"vector" + 0.009*"proof" + 0.008*"theorem" + 0.008*"graph" + 0.007*"network" + 0.007*"simul" + 0.007*"bind"
2023-01-26 18:58:19,719 : INFO : topic #3 (0.103): 0.014*"user" + 0.008*"human" + 0.008*"particip" + 0.006*"media" + 0.006*"game" + 0.006

In [3]:
from gensim.test.utils import datapath
file = datapath("model")
# lda_model.save(file)
lda = gensim.models.LdaModel.load(file)

2023-03-15 22:47:41,238 : INFO : loading LdaModel object from /usr/local/lib/python3.10/site-packages/gensim/test/test_data/model
2023-03-15 22:47:41,245 : INFO : loading expElogbeta from /usr/local/lib/python3.10/site-packages/gensim/test/test_data/model.expElogbeta.npy with mmap=None
2023-03-15 22:47:41,255 : INFO : setting ignored attribute state to None
2023-03-15 22:47:41,255 : INFO : setting ignored attribute dispatcher to None
2023-03-15 22:47:41,256 : INFO : setting ignored attribute id2word to None
2023-03-15 22:47:41,257 : INFO : LdaModel lifecycle event {'fname': '/usr/local/lib/python3.10/site-packages/gensim/test/test_data/model', 'datetime': '2023-03-15T22:47:41.257725', 'gensim': '4.3.0', 'python': '3.10.6 (main, Aug 30 2022, 05:12:36) [Clang 13.1.6 (clang-1316.0.21.2.5)]', 'platform': 'macOS-12.5.1-x86_64-i386-64bit', 'event': 'loaded'}
2023-03-15 22:47:41,258 : INFO : loading LdaState object from /usr/local/lib/python3.10/site-packages/gensim/test/test_data/model.state

## Get Topic Names

In [7]:
x=lda.show_topics(num_topics=20, num_words=5,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Topics and Words
for topic,words in topics_words:
    print(str(topic)+ "::"+ str(words))
print()

#Below Code Prints Only Words 
for topic,words in topics_words:
    print(str(topic) + "," + ",".join(words))
    
# topic name are top 5 representative words, cleaned to be human readible

0::['languag', 'sentenc', 'verb', 'semant', 'argument']
1::['reaction', 'chem', 'oxid', 'acid', 'chemic']
2::['stress', 'strain', 'film', 'particl', 'layer']
3::['glyph', 'user', 'human', 'particip', 'game']
4::['compani', 'industri', 'custom', 'manufactur', 'busi']
5::['optic', 'beam', 'laser', 'magnet', 'puls']
6::['water', 'ocean', 'wind', 'climat', 'environment']
7::['veloc', 'wave', 'boundari', 'pressur', 'frequenc']
8::['govern', 'polit', 'polici', 'countri', 'social']
9::['user', 'network', 'file', 'code', 'messag']
10::['citi', 'hous', 'urban', 'land', 'street']
11::['vehicl', 'aircraft', 'robot', 'sensor', 'flight']
12::['algorithm', 'matrix', 'vector', 'random', 'proof']
13::['devic', 'voltag', 'signal', 'frequenc', 'circuit']
14::['rock', 'fault', 'mantl', 'zone', 'depth']
15::['patient', 'health', 'school', 'worker', 'popul']
16::['imag', 'train', 'detect', 'cluster', 'signal']
17::['cell', 'protein', 'gene', 'bind', 'sequenc']
18::['temperatur', 'heat', 'pressur', 'fuel', 

## Evaluation

In [16]:
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_data_filepath = os.path.join('./ldavis')

# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = gensimvis.prepare(lda_model, bow, dictionary)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared, './ldavis.html')

LDAvis_prepared

/usr/local/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from im

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
17    -0.052758 -0.071355       1        1  8.210960
10     0.164588 -0.103333       2        1  7.398227
8      0.237938 -0.073318       3        1  6.553331
19     0.190706 -0.086082       4        1  6.151565
4      0.185695 -0.007594       5        1  6.131996
12    -0.007722  0.187439       6        1  5.976096
5     -0.184157  0.044293       7        1  5.675631
3      0.148715  0.065733       8        1  5.342032
2     -0.200754 -0.052050       9        1  5.283517
9      0.070458  0.185084      10        1  5.258209
1     -0.156310 -0.145083      11        1  4.814377
11    -0.043300  0.108916      12        1  4.717096
7     -0.177750  0.076686      13        1  4.622114
16    -0.034105  0.159584      14        1  4.406689
18    -0.152979 -0.101735      15        1  4.278927
13    -0.144475  0.114599      16        1  4.047891
6     -0.047151 -0.196533      17        1  3.588896
15     0.193898 -0.050558      18        1  3.054123
0      0.116219  0.097123      19        1  2.279717
14    -0.106755 -0.151815      20        1  2.208607, topic_info=            Term          Freq         Total Category  logprob  loglift
2198        cell  1.568606e+06  1.568606e+06  Default  30.0000  30.0000
1456       water  9.226270e+05  9.226270e+05  Default  29.0000  29.0000
1758        imag  8.934250e+05  8.934250e+05  Default  28.0000  28.0000
2080  temperatur  1.050887e+06  1.050887e+06  Default  27.0000  27.0000
218         citi  7.281440e+05  7.281440e+05  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
4063       ocean  3.435850e+04  1.703154e+05  Topic20  -5.4136   2.2120
1868       north  2.648453e+04  1.694578e+05  Topic20  -5.6739   1.9568
4218      tunnel  2.387445e+04  6.999324e+04  Topic20  -5.7777   2.7372
2080  temperatur  2.835179e+04  1.050887e+06  Topic20  -5.6058   0.2001
2240     deposit  2.506035e+04  1.939964e+05  Topic20  -5.7292   1.7663

[1825 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
5655       9  0.999803        abaqu
2787       1  0.001599      absorpt
2787       2  0.008404      absorpt
2787       3  0.000035      absorpt
2787       4  0.018464      absorpt
...      ...       ...          ...
8172      15  0.116981         zone
8172      16  0.000599         zone
8172      17  0.060316         zone
8172      20  0.227883         zone
54888     17  0.999725  zooplankton

[10675 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 11, 9, 20, 5, 13, 6, 4, 3, 10, 2, 12, 8, 17, 19, 14, 7, 16, 1, 15])

In [17]:
columns = ["filename"]
for i in range(20): columns.append("topic_" + str(i))
df = pd.DataFrame(columns=columns)

In [18]:
files = [x for x in os.listdir("./full_text_processed") if ".pdf.txt" in x]

for fname, doc in zip(files, bow):
    row = {"filename": fname}
    for topic, probability in lda_model.get_document_topics(doc, minimum_probability=0):
       row["topic_" + str(topic)] = probability
    df = df.append(row, ignore_index=True)

print(df.shape)
df.head()

(58520, 21)


,filename,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,759082635-MIT.pdf.txt,0.000006,0.000006,0.000009,0.150197,0.003009,0.000008,0.007480,0.000012,0.193896,...,0.086814,0.000012,0.001240,0.000011,0.000005,0.554907,0.000012,0.000005,0.000010,0.002350
1,1191837094-MIT.pdf.txt,0.000006,0.000006,0.226521,0.109190,0.026208,0.000009,0.000009,0.022014,0.000012,...,0.000013,0.430697,0.000016,0.060527,0.000005,0.000011,0.007887,0.000006,0.011677,0.000010
2,41567358-MIT.pdf.txt,0.024347,0.000014,0.014095,0.101195,0.006755,0.000021,0.000019,0.034712,0.000026,...,0.000028,0.000029,0.024684,0.000026,0.000011,0.000023,0.529282,0.002177,0.000025,0.008134
3,54809434-MIT.pdf.txt,0.027965,0.028368,0.045288,0.074553,0.069981,0.042054,0.038833,0.058664,0.053850,...,0.058142,0.059489,0.073607,0.053643,0.023248,0.047499,0.057960,0.026151,0.051665,0.047150
4,858871740-MIT.pdf.txt,0.000015,0.000015,0.000024,0.000040,0.017980,0.000022,0.000021,0.000031,0.000029,...,0.015130,0.114154,0.201801,0.073195,0.000012,0.000025,0.000466,0.000014,0.000027,0.000025


In [20]:
df.to_csv("embeddings.csv")

## Project to 2D

In [21]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
tsne_results = tsne.fit_transform(df.copy().drop("filename", axis=1).values)

In [22]:
tsne_df = pd.DataFrame(tsne_results, columns = ["tsne_0", "tsne_1"])
tsne_df.head()

,tsne_0,tsne_1
0,57.670830,-13.101546
1,-20.449951,-5.929507
2,-7.268325,-56.778572
3,8.045418,57.828835
4,7.966490,-72.176842


In [23]:
projected_df = pd.concat([df, tsne_df], axis=1)
projected_df.head()

,filename,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,tsne_0,tsne_1
0,759082635-MIT.pdf.txt,0.000006,0.000006,0.000009,0.150197,0.003009,0.000008,0.007480,0.000012,0.193896,...,0.001240,0.000011,0.000005,0.554907,0.000012,0.000005,0.000010,0.002350,57.670830,-13.101546
1,1191837094-MIT.pdf.txt,0.000006,0.000006,0.226521,0.109190,0.026208,0.000009,0.000009,0.022014,0.000012,...,0.000016,0.060527,0.000005,0.000011,0.007887,0.000006,0.011677,0.000010,-20.449951,-5.929507
2,41567358-MIT.pdf.txt,0.024347,0.000014,0.014095,0.101195,0.006755,0.000021,0.000019,0.034712,0.000026,...,0.024684,0.000026,0.000011,0.000023,0.529282,0.002177,0.000025,0.008134,-7.268325,-56.778572
3,54809434-MIT.pdf.txt,0.027965,0.028368,0.045288,0.074553,0.069981,0.042054,0.038833,0.058664,0.053850,...,0.073607,0.053643,0.023248,0.047499,0.057960,0.026151,0.051665,0.047150,8.045418,57.828835
4,858871740-MIT.pdf.txt,0.000015,0.000015,0.000024,0.000040,0.017980,0.000022,0.000021,0.000031,0.000029,...,0.201801,0.073195,0.000012,0.000025,0.000466,0.000014,0.000027,0.000025,7.966490,-72.176842


In [24]:
projected_df.to_csv("projected.csv")

Let's merge with metadata and save it

In [29]:
metadata = pd.read_csv("data/metadata.csv")
print(metadata.shape)
metadata.head()

(57777, 13)


,Unnamed: 0,author,department,issued,uri,oclc,abstract,subject,title,image_url,filename,handle,identifier
0,0,"Vázquez, Maribel, 1971-",Massachusetts Institute of Technology. Dept. o...,2001,http://hdl.handle.net/1721.1/8302,5.044436e+07,Electrophoresis of DNA has become particularly...,Mechanical Engineering.,A study of loading parameters that affect DNA ...,https://mit.atmire.com/bitstream/1721.1/8302/5...,50444365-MIT.pdf.txt,1721.1/8302,oai:mit.atmire.com:1721.1/8302
1,1,"Dai, Siyu, S.M. Massachusetts Institute of Tec...",Massachusetts Institute of Technology. Departm...,2018,http://hdl.handle.net/1721.1/120230,1.083120e+09,"For high-dimensional robots, motion planning i...",Mechanical Engineering.,Probabilistic motion planning and optimization...,https://mit.atmire.com/bitstream/1721.1/120230...,1083120469-MIT.pdf.txt,1721.1/120230,oai:mit.atmire.com:1721.1/120230
2,2,"Bury, Mark Eric",Massachusetts Institute of Technology. Dept. o...,1997,http://hdl.handle.net/1721.1/50310,3.789658e+07,NaN,Aeronautics and Astronautics,Influence of Reynolds number and blade geometr...,https://mit.atmire.com/bitstream/1721.1/50310/...,37896579-MIT.pdf.txt,1721.1/50310,oai:mit.atmire.com:1721.1/50310
3,3,"Meng, Xianglin, S.M. Massachusetts Institute o...",Massachusetts Institute of Technology. Departm...,2018,http://hdl.handle.net/1721.1/117814,1.051460e+09,Our goal is to understand the functioning of t...,Electrical Engineering and Computer Science.,Systemic risk in the interbank lending market,https://mit.atmire.com/bitstream/1721.1/117814...,1051460349-MIT.pdf.txt,1721.1/117814,oai:mit.atmire.com:1721.1/117814
4,4,"Verma, Malvika.",Massachusetts Institute of Technology. Departm...,2019,https://hdl.handle.net/1721.1/123066,1.127292e+09,Lack of medication adherence is a worldwide pr...,Biological Engineering.,Gastric resident systems for large dose drug d...,https://dspace.mit.edu/bitstream/1721.1/123066...,1127292014-MIT.pdf.txt,1721.1/123066,oai:dspace.mit.edu:1721.1/123066


In [31]:
merged = projected_df.merge(metadata, how='inner', on="filename")


In [32]:
def get_main_topic(row):
    max_value = 0
    main_topic = None
    for i in range(20):
        if row["topic_" + str(i)] > max_value: 
            max_value = row["topic_" + str(i)]
            main_topic = i
    
    return main_topic
        
merged['main_topic'] = merged.apply(get_main_topic, axis = 1)

In [33]:
merged = merged.drop('Unnamed: 0', axis=1)

In [34]:
print(merged.shape)
merged.head()

(57763, 35)


,filename,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,issued,uri,oclc,abstract,subject,title,image_url,handle,identifier,main_topic
0,759082635-MIT.pdf.txt,0.000006,0.000006,0.000009,0.150197,0.003009,0.000008,0.007480,0.000012,0.193896,...,2011,http://hdl.handle.net/1721.1/67217,7.590826e+08,"Personal teacher self-efficacy (PTE), or the b...",Urban Studies and Planning.,Teacher self-efficacy in Cape Town : a bottom ...,https://mit.atmire.com/bitstream/1721.1/67217/...,1721.1/67217,oai:mit.atmire.com:1721.1/67217,15
1,1191837094-MIT.pdf.txt,0.000006,0.000006,0.226521,0.109190,0.026208,0.000009,0.000009,0.022014,0.000012,...,2020,https://hdl.handle.net/1721.1/127125,1.191837e+09,"In the age of automation and computation, smar...",Mechanical Engineering.,Design of dynamically controlled desktop fiber...,https://dspace.mit.edu/bitstream/1721.1/127125...,1721.1/127125,oai:dspace.mit.edu:1721.1/127125,11
2,41567358-MIT.pdf.txt,0.024347,0.000014,0.014095,0.101195,0.006755,0.000021,0.000019,0.034712,0.000026,...,1998,http://hdl.handle.net/1721.1/46227,4.156736e+07,NaN,Electrical Engineering and Computer Science,A study of improvements for the Winbank Courte...,https://mit.atmire.com/bitstream/1721.1/46227/...,1721.1/46227,oai:mit.atmire.com:1721.1/46227,16
3,54809434-MIT.pdf.txt,0.027965,0.028368,0.045288,0.074553,0.069981,0.042054,0.038833,0.058664,0.053850,...,2003,http://hdl.handle.net/1721.1/7976,5.480943e+07,Microchips for constant release are not a new ...,Materials Science and Engineering.,Controlled release microchip,https://mit.atmire.com/bitstream/1721.1/7976/5...,1721.1/7976,oai:mit.atmire.com:1721.1/7976,3
4,858871740-MIT.pdf.txt,0.000015,0.000015,0.000024,0.000040,0.017980,0.000022,0.000021,0.000031,0.000029,...,2013,http://hdl.handle.net/1721.1/81609,8.588717e+08,One of the biggest challenges in implementing ...,Mechanical Engineering.,Building blocks for co-design of controllers a...,https://mit.atmire.com/bitstream/1721.1/81609/...,1721.1/81609,oai:mit.atmire.com:1721.1/81609,9


In [35]:
merged = merged.drop_duplicates(subset=['filename'])

In [36]:
merged.to_csv("merged.csv", index = False)

## Similarity

In [4]:
from gensim.test.utils import datapath
file = datapath("model")
lda = gensim.models.LdaModel.load(file)

2023-01-29 16:55:02,707 : INFO : loading LdaModel object from /usr/local/lib/python3.10/site-packages/gensim/test/test_data/model
2023-01-29 16:55:02,713 : INFO : loading expElogbeta from /usr/local/lib/python3.10/site-packages/gensim/test/test_data/model.expElogbeta.npy with mmap=None
2023-01-29 16:55:02,722 : INFO : setting ignored attribute state to None
2023-01-29 16:55:02,724 : INFO : setting ignored attribute dispatcher to None
2023-01-29 16:55:02,724 : INFO : setting ignored attribute id2word to None
2023-01-29 16:55:02,726 : INFO : LdaModel lifecycle event {'fname': '/usr/local/lib/python3.10/site-packages/gensim/test/test_data/model', 'datetime': '2023-01-29T16:55:02.726101', 'gensim': '4.3.0', 'python': '3.10.6 (main, Aug 30 2022, 05:12:36) [Clang 13.1.6 (clang-1316.0.21.2.5)]', 'platform': 'macOS-12.5.1-x86_64-i386-64bit', 'event': 'loaded'}
2023-01-29 16:55:02,726 : INFO : loading LdaState object from /usr/local/lib/python3.10/site-packages/gensim/test/test_data/model.state

In [9]:
files = pd.read_csv("./data/merged.csv")["filename"]
files

0                                759082635-MIT.pdf.txt
1                               1191837094-MIT.pdf.txt
2                                 41567358-MIT.pdf.txt
3                                 54809434-MIT.pdf.txt
4                                858871740-MIT.pdf.txt
                             ...                      
57745                             35651580-MIT.pdf.txt
57746                             24369363-MIT.pdf.txt
57747    Rahman-r_rahman-meng-eecs-2021-thesis.pdf.txt
57748                             06798349-MIT.pdf.txt
57749                             33431729-MIT.pdf.txt
Name: filename, Length: 57750, dtype: object

In [15]:
def get_distribution(filename):
    f = open("./full_text/" + filename, "r")
    doc = dictionary.doc2bow(preprocess(f.read()))
    return lda[doc]

In [19]:
distributions = {}

for i, f in enumerate(files): 
    print(i, end = "\r")
    distributions[f] = get_distribution(f)

distributions

{'759082635-MIT.pdf.txt': [(3, 0.15019587),
  (8, 0.19389541),
  (10, 0.08681429),
  (15, 0.5549072)],
 '1191837094-MIT.pdf.txt': [(2, 0.22652058),
  (3, 0.10919002),
  (4, 0.02620765),
  (7, 0.022014145),
  (9, 0.105177365),
  (11, 0.43069622),
  (13, 0.060527254),
  (18, 0.011677043)],
 '41567358-MIT.pdf.txt': [(0, 0.024347374),
  (2, 0.014095544),
  (3, 0.10119582),
  (7, 0.034712434),
  (9, 0.25439563),
  (12, 0.024682155),
  (16, 0.5292826)],
 '54809434-MIT.pdf.txt': [(0, 0.027965069),
  (1, 0.028367843),
  (2, 0.045288246),
  (3, 0.07455332),
  (4, 0.069980666),
  (5, 0.04205425),
  (6, 0.038832646),
  (7, 0.058664095),
  (8, 0.053849716),
  (9, 0.061891023),
  (10, 0.05814212),
  (11, 0.059488874),
  (12, 0.07360728),
  (13, 0.05364257),
  (14, 0.023247546),
  (15, 0.04749855),
  (16, 0.05795965),
  (17, 0.026151326),
  (18, 0.051664863),
  (19, 0.047150373)],
 '858871740-MIT.pdf.txt': [(4, 0.017980108),
  (9, 0.5769669),
  (10, 0.015129614),
  (11, 0.11414989),
  (12, 0.2017968

In [27]:
rows = []

for i, f1 in enumerate(files):
    print(i, end = "\r")
    
    dists = []
    for f2 in files: 
        if not f1 == f2:
            dist = gensim.matutils.hellinger(distributions[f1], distributions[f2])
            dists.append([f2, dist])
    
    dists.sort(key=lambda x: x[1])
    
    most_similar = dists[:10]
    row = [f1] + [x[0] for x in most_similar]
    rows.append(row)

In [28]:
similarity = pd.DataFrame(rows, columns = ["filename"] + ["similar_" + str(i) for i in range(10)])
similarity.head()

,filename,similar_0,similar_1,similar_2,similar_3,similar_4,similar_5,similar_6,similar_7,similar_8,similar_9
0,759082635-MIT.pdf.txt,589272652-MIT.pdf.txt,32765450-MIT.pdf.txt,18998569-MIT.pdf.txt,436229832-MIT.pdf.txt,971245412-MIT.pdf.txt,39925754-MIT.pdf.txt,958277850-MIT.pdf.txt,27320643-MIT.pdf.txt,21914395-MIT.pdf.txt,29465539-MIT.pdf.txt
1,1191837094-MIT.pdf.txt,jaeger-amjaeger-MS-MAS-2021.pdf.txt,62860988-MIT.pdf.txt,870532070-MIT.pdf.txt,33063204-MIT.pdf.txt,898210793-MIT.pdf.txt,314404390-MIT.pdf.txt,larocca-alarocca-ms-meche-2022-thesis.pdf.txt,964527415-MIT.pdf.txt,35651645-MIT.pdf.txt,57588498-MIT.pdf.txt
2,41567358-MIT.pdf.txt,1014123603-MIT.pdf.txt,520861836-MIT.pdf.txt,31359674-MIT.pdf.txt,49676393-MIT.pdf.txt,768764405-MIT.pdf.txt,40462834-MIT.pdf.txt,38198486-MIT.pdf.txt,15135710-MIT.pdf.txt,60678787-MIT.pdf.txt,755091768-MIT.pdf.txt
3,54809434-MIT.pdf.txt,25315232-MIT.pdf.txt,20713350-MIT.pdf.txt,29368568-MIT.pdf.txt,37950137-MIT.pdf.txt,37520352-MIT.pdf.txt,28714352-MIT.pdf.txt,03971889-MIT.pdf.txt,27031137-MIT.pdf.txt,04206932-MIT.pdf.txt,23106890-MIT.pdf.txt
4,858871740-MIT.pdf.txt,289332268-MIT.pdf.txt,864014847-MIT.pdf.txt,40265649-MIT.pdf.txt,871165990.pdf.txt,832740002-MIT.pdf.txt,51686626-MIT.pdf.txt,681898520-MIT.pdf.txt,710365774-MIT.pdf.txt,51626122-MIT.pdf.txt,868826451-MIT.pdf.txt


In [29]:
similarity.to_csv("./data/similarity.csv")